In [2]:
from operations import *
from reductions import *
from link_compile import CpuReduc
import time

In [8]:
red_formula_string = "Sum_Reduction((Exp(Minus(Sum(Square((Var(0,3,0) - Var(1,3,1))*Var(1,3,1))))) * Var(2,1,1)),0)"
red_formula = eval(red_formula_string)
print(type(red_formula))
print(red_formula)
print()
print(red_formula.formula)
print(type(red_formula.formula))
print()
print(red_formula._Vars)

<class 'reductions.Sum_Reduction'>
Sum_Reduction(Exp(-Sum(((Var(0,3,0)-Var(1,3,1))*Var(1,3,1))**2))*Var(2,1,1),0)

Exp(-Sum(((Var(0,3,0)-Var(1,3,1))*Var(1,3,1))**2))*Var(2,1,1)
<class 'operations.Mult_'>

{Var(2,1,1), Var(1,3,1), Var(0,3,0)}


In [12]:
myred = CpuReduc(red_formula,dtype="float",dtypeacc="double",nargs=3)

code = myred.get_code()
print(code)

#include <cmath>

#ifdef USE_OPENMP
#include <omp.h>
#endif

extern "C" int CpuReduc(int nx, int ny, float* out , float* arg0, float* arg1, float* arg2) {
    float* args[3];
    args[0] = arg0;
args[1] = arg1;
args[2] = arg2;

    
    
    #pragma omp parallel for
    for (int i = 0; i < nx; i++) {
        float fout[1];
        float xi[3];
        float yj[4];
        double acc[1];
        xi[0] = args[0][i*3+0];
xi[1] = args[0][i*3+1];
xi[2] = args[0][i*3+2];

        #pragma unroll
for(int k=0; k<1; k++)
    acc[k] = (double)(0.0f);
        for (int j = 0; j < ny; j++) {
            yj[0] = args[2][j*1+0];
yj[1] = args[1][j*3+0];
yj[2] = args[1][j*3+1];
yj[3] = args[1][j*3+2];

            
{
// Starting code block for Exp(-Sum(((Var(0,3,0)-Var(1,3,1))*Var(1,3,1))**2))*Var(2,1,1).

float out_exp_3[1];

{
// Starting code block for Exp(-Sum(((Var(0,3,0)-Var(1,3,1))*Var(1,3,1))**2)).

float out_minus_3[1];

{
// Starting code block for -Sum(((Var(0,3,0)-Var(1,3,1))*Var(1,3,1))**2)

In [13]:
myred.compile_code()

In [2]:
start = time.time()
red_formula_string = "Sum_Reduction((Exp(Minus(Sum(Square((Var(0,3,0) - Var(1,3,1))*Var(1,3,1))))) * Var(2,1,1)),0)"
red_formula = eval(red_formula_string)
code = CpuReduc(red_formula,dtype="float",dtypeacc="double")()
f = open("test.cpp","w")
f.write(code)
f.close()
elapsed = time.time()-start
print("code generation time : ", elapsed)

start = time.time()
!g++ -c -O3 -DUSE_OPENMP=1 test.cpp
elapsed = time.time()-start
print("compilation time : ", elapsed)

code generation time :  0.002454996109008789
compilation time :  0.3233649730682373


In [3]:
!clang-format -i test.cpp
f = open("test.cpp")
code = f.read()
f.close()
print(code)

#include <cmath>

#ifdef USE_OPENMP
#include <omp.h>
#endif

extern "C" int CpuReduc(int nx, int ny, float *out, float **args) {

#pragma omp parallel for
  for (int i = 0; i < nx; i++) {
    float fout[1];
    float xi[3];
    float yj[4];
    double acc[1];
    xi[0] = args[0][i * 3 + 0];
    xi[1] = args[0][i * 3 + 1];
    xi[2] = args[0][i * 3 + 2];

#pragma unroll
    for (int k = 0; k < 1; k++)
      acc[k] = (double)(0.0f);
    for (int j = 0; j < ny; j++) {
      yj[0] = args[2][j * 1 + 0];
      yj[1] = args[1][j * 3 + 0];
      yj[2] = args[1][j * 3 + 1];
      yj[3] = args[1][j * 3 + 2];

      {
        // Starting code block for
        // Mult(Exp(Minus(Sum(Square(Mult(Subtract(Var(0,3,0),Var(1,3,1)),Var(1,3,1)))))),Var(2,1,1)).

        float out_exp_1[1];

        {
          // Starting code block for
          // Exp(Minus(Sum(Square(Mult(Subtract(Var(0,3,0),Var(1,3,1)),Var(1,3,1)))))).

          float out_minus_1[1];

          {
            // Starting code block f

In [4]:
red_formula_string = "Grad_WithSavedForward(Sum_Reduction((Exp(Minus(Sum(Square((Var(0,3,0) - Var(1,3,1))))))),0), Var(0,3,0), Var(3,1,0), Var(4,1,0))"
red_formula = eval(red_formula_string)
print(red_formula)

Sum_Reduction
    Minus
        Mult
            IntConstant(2)
            Mult
                Subtract
                    Var(0,3,0)
                    Var(1,3,1)
                SumT
                    Mult
                        Exp
                            Minus
                                Sum
                                    Square
                                        Subtract
                                            Var(0,3,0)
                                            Var(1,3,1)
                        Var(3,1,0)
